In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
import random
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch
import transformers


In [ ]:
!pip install transformers

In [ ]:
def sample(data):
  all_user = set(data["userId"])
  user_cnt = data["userId"].value_counts().to_dict()

  train, val = pd.DataFrame(columns=['userId', 'movieId', 'rating', 'timestamp', 'datetime'], dtype=object), pd.DataFrame(columns=['userId', 'movieId', 'rating', 'timestamp', 'datetime'], dtype=object)
  for user in all_user:
    count = user_cnt[user]
    sub = data[data['userId'] == user]
    sub_train, sub_val = train_test_split(sub, test_size=0.2, random_state=2022)
    train = train.append(sub_train, ignore_index=True)
    val = val.append(sub_val, ignore_index=True)
  # print(train)
  # print(val)
  return train, val

In [ ]:

path = 'small'
ratings = pd.read_csv(os.path.join(path, "ratings.csv"), sep=",", nrows=2000)
ratings["datetime"] = ratings["timestamp"].map(lambda x: datetime.fromtimestamp(x))
num_users = len(set(ratings['userId']))
print(ratings.columns)
train_data, val_data = sample(ratings)
print(num_users)

In [ ]:
train_data.head(10)

In [ ]:
val_data.head(10)

In [ ]:
tags = pd.read_csv(os.path.join(path, "tags.csv"), sep=",", nrows=2000)
tags['datetime'] = tags['timestamp'].map(lambda x: datetime.fromtimestamp(x))
tags.head(10)

In [ ]:
movies = pd.read_csv(os.path.join(path, "movies.csv"), sep=",", nrows=None)
movies.head(10)

In [ ]:
import re

pattern = '\(\d\d\d\d\)'

movie_title = movies['title']

# title = movie_title.map(lambda x: re.sub(pattern, " ", x))
# year = movie_title.map(lambda x: re.findall("\d\d\d\d", x)[-1] if re.search("\d\d\d\d", x) is not None else "0000")
# print(year)
movie_year = {}
movie_dict = {}
movieIdset = set(movies["movieId"])
for movieId in movieIdset:
  sub_tags = tags[tags['movieId']==movieId]
  sub_movies = movies[movies['movieId']==movieId]
  if movieId not in movie_dict:
    movie_dict[movieId] = {}
  title = sub_movies['title'].item()
  movie_dict[movieId]['title'] = re.sub(pattern, " ", title)
  movie_dict[movieId]['genres'] = list(sub_movies['genres'])[0].replace("|", " ")
  if len(sub_tags) > 0:
    if "tag" not in movie_dict[movieId]:
      movie_dict[movieId]['tag'] = ""
    movie_dict[movieId]['tag'] += " ".join(list(sub_tags['tag']))
  movie_year[movieId] = int(re.findall("\d\d\d\d", title)[-1]) if re.search("\d\d\d\d", title) is not None else 1800
# print(movie_dict)
# print(movie_year)



In [ ]:
movie_text = {}
for movieId, info in movie_dict.items():
  movie_text[movieId] = info['title'].strip() + " " + info['genres'].strip() + ((" tags "+" ".join(set(info['tag'].strip().split(" ")))) if 'tag' in info else "")
# print(movie_text)
# print(max([len(x.split(" ")) for x in movie_text.values()]))

# word tokenize

In [ ]:
from transformers import BertModel, AdamW, BertConfig, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

input_ids = []
attention_masks = []

movie_token = {}
for movieId, text in movie_dict.items():
  movie_token[movieId] = {}
  encoded_dict = tokenizer.encode_plus(
    text=movie_text[movieId],                      # Sentence to encode.
    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
    max_length = 210,           # Pad & truncate all sentences.
    pad_to_max_length = True,
    return_attention_mask = True,   # Construct attn. masks
    return_tensors = 'pt',     # Return pytorch tensors.
  )
  movie_token[movieId]['input_ids'] = encoded_dict['input_ids']
  movie_token[movieId]['attention_mask'] = encoded_dict['attention_mask']

# print(movie_token[1])


# Dataset, LatentFactorModel




In [ ]:
from transformers import BertModel, AdamW, BertConfig, BertTokenizer
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu"

print(device)


class MyDataset(Dataset):
  """
    x: [userId, movieId, rating, timestamp, datetime]
  """
  def __init__(self, df_sample, token_dict, year_dict):
    super(MyDataset, self).__init__()
    self.model = BertModel.from_pretrained("bert-base-uncased")
    self.model.to(device)
    self.df_sample = df_sample.values
    # print(self.df_sample)
    self.token_dict = token_dict
    self.year_dict = year_dict
  

  def __len__(self):
    # print(self.df_sample)
    return len(self.df_sample)

  # return cls, userId, year-diff, label
  def __getitem__(self, idx):
    # print(idx, len(self.df_sample))
    sample = self.df_sample[idx]
    userId, movieId, score, time = sample[0], sample[1], sample[2], sample[4]
    token = self.token_dict[movieId]
    input_ids, attention_mask = token['input_ids'].to(device), token['attention_mask'].to(device)
    CLS = self.model(input_ids, token_type_ids=None, attention_mask=attention_mask)[1]
    # print(CLS.shape)
    year_diff = time.year - self.year_dict[movieId]
    return CLS.to(device), userId, year_diff, score

# LatentFactorModel

In [ ]:
class LatentFactorModel(nn.Module):
    def __init__(self, num_users, latent_dim):
        super(LatentFactorModel, self).__init__()
        self.userEmbedding = nn.Embedding(num_users+1, latent_dim)
        self.userEmbedding.weight.data.uniform_(-1, 1)
        self.w_year = Variable(torch.tensor(0.5), requires_grad=True).to(device)
        self.w_dot = Variable(torch.tensor(0.5), requires_grad=True).to(device)
        self.bias = Variable(torch.tensor(3.0), requires_grad=True).to(device)
        self.sigmoid = nn.Sigmoid()

    def forward(self, userId, movie_vec, year_diff):
        user_vec = self.userEmbedding(userId)
        user_vec = torch.unsqueeze(user_vec, 1)
        movie_vec = torch.permute(movie_vec, (0, 2, 1))
        ret = (self.sigmoid(torch.bmm(user_vec, movie_vec).reshape(-1))+(1/9))*4.5 # + self.bias #+ self.w_year * year_diff 
        return ret

# train model

In [ ]:
from torch.nn.modules.loss import MSELoss
def train(data, batch_size=16, shuffle=True, lr=0.001, num_epoch=100, movie_dict={}, movie_year={}, latent_dim=768):
  train_data, val_data = data[0], data[1]
  train_loader = DataLoader(MyDataset(train_data, movie_dict, movie_year), batch_size=batch_size, shuffle=shuffle)

  model = LatentFactorModel(num_users=num_users, latent_dim=latent_dim)
  model.to(device)
  model.train()
  optimizer = torch.optim.SGD(params=model.parameters(), lr=lr, weight_decay=0.01)
#   optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr, weight_decay=0.01)
#   optimizer = nn.DataParallel(optimizer, device_ids=device_ids)
#   optimizer.to(device)


  mse_loss = nn.MSELoss()
  # crossEntropy = nn.CrossEntropyLoss()
  for i in range(num_epoch):
    print("================{i} iteration================ ".format(i=i + 1))
    predList = np.array([])
    labelList = np.array([])
    for j, sample in enumerate(train_loader):
      CLS, userId, year_diff, score = sample[0].to(device), sample[1].to(device), sample[2].to(device), sample[3].to(device)
      optimizer.zero_grad()
      pred = model(userId, CLS, year_diff)
      # print(pred, score)
      loss = mse_loss(pred, score.float())
      predList = np.append(predList, pred.detach().cpu().numpy())
      labelList = np.append(labelList, score.detach().cpu().numpy())
      loss.backward()
      optimizer.step()
      if (j+1)%500 == 0:
        print(loss.item())
    total_loss = mse_loss(torch.tensor(predList), torch.tensor(labelList))
    print("training loss={loss}".format(loss=loss))
    model.eval()
    eval(val_data, model, batch_size, shuffle, movie_dict, movie_year, latent_dim)
    model.train()
    if (i)%20 == 0:
      state = {
        'epoch': (i+1),
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
      }
      filepath = "lfm_epoch{epoch}".format(epoch=(i+1))
      torch.save(state, "./model/{filepath}.pt".format(filepath=filepath))
      # state = torch.load(filepath)
      # model.load_state_dict(state['state_dict'])
      # optimizer.load_state_dict(state['optimizer'])
      

# Eval

In [ ]:
def eval(val_data, model, batch_size=1, shuffle=True, movie_dict={}, movie_year={}, latent_dim=768):
  val_loader = DataLoader(MyDataset(val_data, movie_dict, movie_year), batch_size=batch_size, shuffle=shuffle)

  mse_loss = nn.MSELoss()
  # crossEntropy = nn.CrossEntropyLoss()
  predList = np.array([])
  labelList = np.array([])
  for j, sample in enumerate(val_loader):
    CLS, userId, year_diff, score = sample[0].to(device), sample[1].to(device), sample[2].to(device), sample[3].to(device)
    pred = model(userId, CLS, year_diff)
#    print(pred, score)
    predList = np.append(predList, pred.detach().cpu().numpy())
    labelList = np.append(labelList, score.detach().cpu().numpy())

  val_loss = mse_loss(torch.tensor(predList), torch.tensor(labelList))
  print("validate loss={loss}".format(loss=val_loss))


In [ ]:
train(data=(train_data, val_data), batch_size=1, shuffle=True, lr=0.001, num_epoch=1, movie_dict=movie_token, movie_year=movie_year, latent_dim=768)

In [ ]:
movieSet = set(movies['movieId'])
tagMovieSet = set(tags['movieId'])
ratingMovieSet = set(ratings['movieId'])
print(len(movieSet), len(tagMovieSet), len(ratingMovieSet))
print(len(movieSet.intersection(tagMovieSet)))
print(len(movieSet.intersection(ratingMovieSet)))
print(len(ratingMovieSet.intersection(tagMovieSet)))
a = torch.zeros(2,3).to(device)

In [ ]:
!nvidia-smi -l 10